In [1]:
from keras import preprocessing
from sklearn.preprocessing import LabelEncoder
from keras.layers import concatenate
import tensorflow.keras as tfk
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, cohen_kappa_score
from keras.utils import to_categorical
import keras
import timeit
import random
from pathlib import Path
from sklearn.utils import shuffle
import time
from sklearn.utils.extmath import softmax
import tensorflow as tf
import sys
from tqdm import tqdm
import os
import shutil
import geopandas as gpd
import rasterio

ModuleNotFoundError: No module named 'keras'

In [ ]:

# Define a function to download images from the provided date range
def download_images(start_date, end_date):
    setup_logging(1)  # 0: nothing, 1: only progress bars, 2: INFO, 3: DEBUG

    dag = EODataAccessGateway()
    geometry = "POLYGON ((8.657227 36.958086, 8.97583 37.001968, 8.986816 36.729493, 9.085693 36.491385, 8.789063 36.411852, 8.371582 36.385323, 8.129883 36.464883, 8.360596 36.579658, 8.415527 36.755904, 8.613281 36.843875, 8.657227 36.958086))"
    search_results, total_count = dag.search(
        productType="S2_MSI_L1C",
        geom=geometry,
        start=start_date,
        end=end_date,
        cloudCover=4,
    )

    products_to_download = search_results[:2]  # Download 2 images, change as needed
    paths = dag.download_all(products_to_download)
    for file_path in paths:
        destination_path = os.path.join(
            "sentinel2", os.path.basename(file_path)
        )
        shutil.move(file_path, destination_path)

# Download images for the specified date ranges
date_ranges = [
    ("2022-01-20", "2022-02-23"),
    ("2023-11-13", "2023-11-15")
]

for start_date, end_date in date_ranges:
    download_images(start_date, end_date)


In [ ]:
image_paths = [
    ('sentinel2/T32SMF_20231114T101159_B02.jp2',
     'sentinel2/T32SMF_20231114T101159_B03.jp2'),
    ('sentinel2/T32SMF_20231114T101159_B04.jp2',
     'sentinel2/T32SMF_20231114T101159_B08.jp2')
]

In [ ]:
########## Read Bands for pixel based classification
b2 = rasterio.open('sentinel2/T32SMF_20231114T101159_B02.jp2')
b2_array = b2.read()
b2_array = np.moveaxis(b2_array,0,-1)
b2_array = (b2_array - np.percentile(b2_array, 2) )/ (np.percentile(b2_array, 98) - np.percentile(b2_array, 2) )


b3 = rasterio.open('sentinel2/T32SMF_20231114T101159_B03.jp2')
b3_array = b3.read()
b3_array = np.moveaxis(b3_array,0,-1)
b3_array = (b3_array - np.percentile(b3_array, 2) )/ (np.percentile(b3_array, 98) - np.percentile(b3_array, 2) )


b4 = rasterio.open('sentinel2/T32SMF_20231114T101159_B04.jp2')
b4_array = b4.read()
b4_array = np.moveaxis(b4_array,0,-1)
b4_array = (b4_array - np.percentile(b4_array, 2) )/ (np.percentile(b4_array, 98) - np.percentile(b4_array, 2) )


b8 = rasterio.open('sentinel2/T32SMF_20231114T101159_B08.jp2')
b8_array = b8.read()
b8_array = np.moveaxis(b8_array,0,-1)
b8_array = (b8_array - np.percentile(b8_array, 2) )/ (np.percentile(b8_array, 98) - np.percentile(b8_array, 2) )

In [ ]:
# Initialize an empty list to store time series data
time_series_data = []

# Loop through each date
for date_images in image_paths:
    # Initialize an empty list to store band data for the current date
    date_data = []
    
    # Loop through each band for the current date
    for band_path in date_images:
        # Read band data using rasterio
        with rasterio.open(band_path) as src:
            band_data = src.read(1)  # Read band data (adjust band index if needed)
            date_data.append(band_data)  # Append band data for the current date
        
    # Stack the bands for the current date into a 3D array
    date_array = np.stack(date_data, axis=-1)
    
    # Append the 3D array representing the bands for the current date to the time series data list
    time_series_data.append(date_array)

# Stack the 3D arrays for each date into a 4D array representing the entire time series
time_series = np.stack(time_series_data, axis=1)
time_series = time_series.transpose(0, 2, 1, 3).reshape(10980, 10980, 4)

In [ ]:
time_series.shape

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Lire le fichier de formes
shapefile_path = 'Jendouba2.shp'
data = gpd.read_file(shapefile_path)

# Récupérer les géométries des classes
geometries = data['geometry']

# Ouvrir la bande b8 avec rasterio
with rasterio.open('sentinel2/T32SMF_20231114T101159_B08.jp2') as src:
    # Récupérer les dimensions de l'image
    largeur_de_l_image = src.width
    hauteur_de_l_image = src.height
    
# Parcourir les pixels de l'image
for x in range(largeur_de_l_image):
    for y in range(hauteur_de_l_image):
        # Coordonnées spatiales du pixel
        pixel_coords = Point(x, y)
        
        # Vérifier si le pixel appartient à l'un des polygones
        for idx, geometry in enumerate(geometries):
            if geometry.contains(pixel_coords):
                # Assigner la classe correspondante au pixel
                pixel_class = data['class'][idx]  # 'class' est la colonne des classes dans le shapefile
                # Faire quelque chose avec la classe attribuée au pixel


In [ ]:
for x in range(largeur_de_l_image):
    for y in range(hauteur_de_l_image):
        # Coordonnées spatiales du pixel
        pixel_coords = Point(x, y)
        pixel_class = None  # Classe par défaut si le pixel ne correspond à aucun polygone
        
        # Vérifier si le pixel appartient à l'un des polygones
        for idx, geometry in enumerate(geometries):
            if geometry.contains(pixel_coords):
                # Assigner la classe correspondante au pixel
                pixel_class = data['class'][idx]  # 'class' est la colonne des classes dans le shapefile
                break  # Sortir de la boucle une fois la classe trouvée
        
        # Stocker la classe attribuée à ce pixel dans le dictionnaire
        class_assignments[(x, y)] = pixel_class

# Enregistrer les résultats dans un fichier CSV ou JSON
import pandas as pd

# Créer un DataFrame à partir du dictionnaire des classes attribuées aux pixels
df = pd.DataFrame(list(class_assignments.items()), columns=['Pixel_coords', 'Class'])
csv_filename = 'pixel_class_assignments.csv'
df.to_csv(csv_filename, index=False)

Train test split

In [ ]:
# Pixels
    train_X = []
    for band in [b2_array,b3_array,b4_array,b8_array]:
        train_X.append( band[train_idx] )
        X_train = np.stack(train_X,axis=-1)

    valid_X_pxl = []
    for band in [b2_array,b3_array,b4_array,b8_array]:
        valid_X.append( band[valid_idx] )
        X_valid = np.stack(valid_X,axis=-1)

    test_X_pxl = []
    for band in [b2_array,b3_array,b4_array,b8_array]:
        test_X.append( band[test_idx] )
        X_test = np.stack(test_X,axis=-1)

    np.save('train.npy',X_train)
    np.save('test.npy',X_valid)
    np.save('valid',X_test)


In [ ]:
def build_one_branch_model(input_shape, nb_classes):
    n_feature_maps = 128
    dropOut = 0.5

    input_layer = tfk.layers.Input(name="input", shape=input_shape)
    # features = getBranch(input_layer, n_feature_maps, dropOut, "input")
    # dense_layer = tfk.layers.Dense(512, activation='relu')(features)

    dense_layer = tfk.layers.Dense(n_feature_maps, activation='relu')(input_layer)
    dense_layer = tfk.layers.Dropout(rate=dropOut)(dense_layer)
    dense_layer = tfk.layers.Dense(n_feature_maps, activation='relu')(dense_layer)
    output_layer = tfk.layers.Dense(nb_classes, activation='softmax')(dense_layer)

    model = tfk.models.Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
n_classes = len(np.unique(train_y))
model = build_one_branch_model((32), n_classes )
model.summary()

In [ ]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])


In [ ]:
model.fit(train_X, train_y, epochs=num_epochs, batch_size=batch_size, validation_data=(val_X, val_y))


In [ ]:
test_loss, test_accuracy = model.evaluate(test_X, test_y)

In [ ]:
valid_pred = model.predict(valid_X_pxl,batch_size=BATCH_SIZE)
valid_pred = np.argmax(valid_pred,axis=1)
f1_score(valid_y_enc,valid_pred,average='weighted')

In [ ]:
test_pred = model.predict(test_X_pxl,batch_size=BATCH_SIZE)
test_pred = encoder.inverse_transform( np.argmax(test_pred,axis=1) )

print ('Acc:',accuracy_score(test_y,test_pred))
print ('F1:',f1_score(test_y,test_pred,average='weighted'))
print ('F1:',f1_score(test_y,test_pred,average=None))
print ('Kappa:',cohen_kappa_score(test_y,test_pred))